In [ ]:
from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.append( "/".join(current_dir.split('/')[:-3]))

In [ ]:
import tqdm
import pandas as pd
from stacey.cleaning import filter_english, clean
from stacey.path import get_output_file, get_app_dir, get_output_dir

In [ ]:
df_main = pd.read_csv(get_output_file('aurea_smb_aurea_mailessentials.csv'), encoding='utf-8')

In [ ]:
df_main.dropna(subset=['product'], inplace=True)

In [ ]:
tqdm.tqdm.pandas()

In [ ]:
df_main = df_main[filter_english(df_main['comments_new'])]

In [ ]:
df = df_main

In [ ]:
custom_filters=[
    {"regexp": r"^([0-9]{4}-[0-9]{2}-[0-9]{2}\s([0-1][0-9]|2[0-4]):[0-5][0-9]:[0-5][0-9]).*$", "replace": ""},
    {"regexp":r"^(best regards|kind Regards|regards)([^=]+[\s\S])+$", "replace": ""},
    {"regexp": r"[_]+([\s\S]*)[=]", "replace": ""},
    {"regexp":r"IMPORTANT NOTE FOR AGENT[\s\S]+pending.", "replace": ""},
    {"regexp":r"REMINDER TO AGENT[\s\S]+Central Support", "replace":""},
    {"regexp":r"eta for pending.+?([0-9]{4}-([0][1-9]|1[1-2])-([0-2][0-9]|3[0-1]))", "replace":""}]

In [ ]:
df['clean_comments'] = df['comments_new'].progress_apply(lambda text: clean(text, custom_filters))

In [ ]:
df = df[df['clean_comments'].map(len) > 2]

In [ ]:
df.head()

In [ ]:
df.to_csv(get_output_file('clean_aurea_smb_mails_essentials.csv'), index=False)